# Analyse des interventions non facturées

Ce notebook permet d'analyser les interventions non facturées dans Notion.

In [1]:
import pandas as pd
import datetime

from src.notion_module.analyse import from_interventions_to_datas
from src.notion_module.core import query_unbilled_entries, create_invoice_page, mark_as_billed

# Analyse des interventions non facturées (Étape 2)

In [2]:
def etape_2 ():
    """
        Cette fonction permet de récupérer les interventions non facturées
        dans Notion et de les transformer en DataFrame pour analyse.
        Pour ignorer le statut de facturation, on peut passer le dernier paramètre à None
    """
    interventions = query_unbilled_entries("2025-05-01", "2025-09-30", None)

    datas = from_interventions_to_datas(interventions)

    df = pd.DataFrame(datas)

    # Conversion de la date en datetime
    df['Date de début'] = pd.to_datetime(df['Date de début'])

    # Ajout colonne mois
    df['Mois'] = df['Date de début'].dt.to_period('M')

    # Calcul somme à facturer
    df['Somme à facturer'] = df['Nombre heures'] * df['Tarif horaire']

    print("✅ Les données ont été transformées en DataFrame avec succès.")

    return df

dataframes = etape_2()

✅ Les données ont été transformées en DataFrame avec succès.


In [3]:
def by_city(df):
    df_ville = df.groupby('Ville').agg(
        nb_ecoles=('Ecole', 'nunique'),
        nb_heures=('Nombre heures', 'sum'),
        somme_a_facturer=('Somme à facturer', 'sum')
    ).reset_index()

    print("Nombre d'écoles par ville :")
    print(df_ville)

by_city(dataframes)

Nombre d'écoles par ville :
            Ville  nb_ecoles  nb_heures  somme_a_facturer
0            Evry          1       49.5             485.0
1  Noisy-le-Grand          1       63.0             490.0
2           Paris          2       46.0             690.0


In [4]:
def by_school_and_class(df):
    df_ecole_classe = df.groupby(['Ecole', 'Classe']).agg(
        nb_heures=('Nombre heures', 'sum'),
        somme_a_facturer=('Somme à facturer', 'sum')
    ).reset_index()

    print("Nombre d'heures par école et par classe :")
    print(df_ecole_classe)

by_school_and_class(dataframes)

Nombre d'heures par école et par classe :
                 Ecole    Classe  nb_heures  somme_a_facturer
0                  ECE   BTS SIO        3.5              52.5
1                  ECE        L3       26.5             397.5
2               ENSIIE        L3       19.5             150.0
3               ENSIIE  M1 EDWEB       14.0             210.0
4               ENSIIE  M1 Maths       16.0             125.0
5                ESIEE        M1       63.0             490.0
6  NEXA Digital School   BTS SIO       16.0             240.0


In [5]:
def by_month(df):
    # On distingue passé vs futur (par rapport à aujourd'hui)
    aujourdhui = pd.to_datetime(datetime.datetime.today().date())

    df['periode'] = df['Date de début'].apply(lambda d: 'Passé' if d < aujourdhui else 'Futur')

    df_mois_periode = df.groupby(['Mois', 'periode']).agg(
        nb_heures=('Nombre heures', 'sum')
    ).reset_index()

    print("Nombre d'heures par mois et état de la période :")
    print(df_mois_periode)

by_month(dataframes)

Nombre d'heures par mois et état de la période :
      Mois periode  nb_heures
0  2025-05   Passé       14.0
1  2025-06   Futur       35.0
2  2025-06   Passé       28.0
3  2025-07   Futur       50.0
4  2025-08   Futur       31.5


# Création de la page de facture (Étape 3 à 6)

In [6]:
def etape_3():
    """
        Cette fonction permet de créer une page de facture dans Notion
        à partir des données d'interventions non facturées.
    """
    interventions = query_unbilled_entries("2025-01-01", "2025-09-30", None)

    client = "ENSIIE"

    interventions_client = [
        i for i in interventions
        if i["properties"]["Ecole"]["select"]
        and i["properties"]["Ecole"]["select"]["name"] == client
    ]

    # On crée la page de facture
    create_invoice_page(
        client,
        interventions_client,
        "2025-09",
        "Alexandre5",
    )

    print(f"✅ La page de facture du client {client} a été créée avec succès. Il y a {len(interventions_client)} interventions facturées.")

    mark_as_billed(interventions_client)

    print(f"✅ Les interventions du client {client} ont été marquées comme facturées.")

etape_3()

✅ La page de facture du client ENSIIE a été créée avec succès. Il y a 13 interventions facturées.
✅ Les interventions du client ENSIIE ont été marquées comme facturées.
